In [47]:
import os

def read_csv(file_path):
    """Reads a CSV file and returns a list of dictionaries."""
    if not os.path.exists(file_path):
        print("ERR001: File not found -", file_path)
        return []
    
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if not lines:
            print("ERR002: Invalid data format - Empty file")
            return []
        
        headers = lines[0].strip().split(',')
        for line in lines[1:]:
            values = line.strip().split(',')
            if len(values) != len(headers):
                print("ERR002: Invalid data format -", line)
                continue
            data.append(dict(zip(headers, values)))
    return data

In [48]:
def parse_float(value, default=0.0):
    """Converts a value to float safely."""
    try:
        return float(value)
    except ValueError:
        return default

def group_by_month(sensor_data):
    """Groups sensor data by sensor type and month."""
    grouped = {}
    for entry in sensor_data:
        sensor_type = entry['sensor_type']
        date = entry['date'][:7]  # Extract YYYY-MM
        value = parse_float(entry['value'])
        
        key = (sensor_type, date)
        if key not in grouped:
            grouped[key] = []
        grouped[key].append(value)
    return grouped

In [49]:
def calculate_monthly_statistics(sensor_data):
    """Calculates average, max, and min values for each sensor type per month."""
    grouped = group_by_month(sensor_data)
    stats = []
    
    for (sensor_type, month), values in grouped.items():
        avg_value = sum(values) / len(values) if values else 0.0
        max_value = max(values) if values else 0.0
        min_value = min(values) if values else 0.0
        stats.append(f"{sensor_type},{month},{avg_value:.2f},{max_value:.2f},{min_value:.2f}")
    return stats


In [50]:
def identify_outliers(sensor_data, thresholds):
    """Identifies sensor values that exceed defined thresholds."""
    threshold_map = {t['sensor_type']: (parse_float(t['min_threshold']), parse_float(t['max_threshold'])) for t in thresholds}
    outliers = []
    
    for entry in sensor_data:
        sensor_type = entry['sensor_type']
        value = parse_float(entry['value'])
        min_thresh, max_thresh = threshold_map.get(sensor_type, (None, None))
        
        if min_thresh is not None and value < min_thresh:
            outliers.append(f"{entry['date']},{sensor_type},{value},{entry['unit']},{entry['location_id']},Min")
        elif max_thresh is not None and value > max_thresh:
            outliers.append(f"{entry['date']},{sensor_type},{value},{entry['unit']},{entry['location_id']},Max")
    return outliers

In [51]:
def write_csv(file_path, headers, data):
    """Writes data to a CSV file."""
    with open(file_path, 'w') as file:
        file.write(headers + '\n')
        for row in data:
            file.write(row + '\n')



In [52]:
def main():
    """Main function to process sensor data."""
    sensor_data = read_csv(r'C:\Users\AkhilPokuri\Downloads\sensor_data.csv')
    thresholds = read_csv(r'C:\Users\AkhilPokuri\Downloads\thresholds.csv')

    if not sensor_data:
        print("ERR003: Processing error - No sensor data available")
        return

    monthly_stats = calculate_monthly_statistics(sensor_data)
    outliers = identify_outliers(sensor_data, thresholds)

    write_csv(r'C:\Users\AkhilPokuri\Documents\monthly_stats.csv', 
              "sensor_type,month,avg_value,max_value,min_value", monthly_stats)
    
    write_csv(r'C:\Users\AkhilPokuri\Documents\outliers.csv', 
              "date,sensor_type,value,unit,location_id,threshold_exceeded", outliers)

    print("Processing completed. Reports generated.")

if __name__ == "__main__":
    main()

Processing completed. Reports generated.
